# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



## Imports

In [81]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy, copy
from dataclasses import dataclass
import math
import ipdb

## The *Nim* and *Nimply* classes

In [82]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [83]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        print ("Assert: ", row, num_objects, self._rows[row])
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Strategies

#### Random strategy

>A completely random move

In [84]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


#### Gabriele's strategy
> Pick always the maximum possible number of the lowest row


In [85]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


#### Optimal strategy

>Fixed rules based on *nim-sum*

In [86]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


#### EA strategy
> A strategy that can adapt its parameters

In [87]:
# Set of available operations on the state
F = ['+', '-', '*', 'div', 'mod', 'and', 'or', 'xor']

POPULATION_SIZE = 5
NUM_ROWS = 5
NUM_GAMES = 10

In [88]:
def check_result(state: list, nim: Nim, row: int, num: int):
    nim = deepcopy(nim)

    result = state[0]
    
    nim.nimming(Nimply(row, num))

    for i in range(1, len(state), 2):
        operator = state[i]
        n = state[i + 1]

        if operator == "+":
            result = result + nim.rows[n]
        elif operator == "-":
            result = result - nim.rows[n]
        elif operator == "*":
            result = result * nim.rows[n]
        elif operator == "div":
            if nim.rows[n] != 0:
                result = int(result / nim.rows[n])
            else:
                return None
        elif operator == "mod":
            if nim.rows[n] != 0:
                result = result % nim.rows[n]
            else:
                return None
        elif operator == "and":
            result = result & nim.rows[n]
        elif operator == "or":
            result = result | nim.rows[n]
        elif operator == "xor":
            result = result ^ nim.rows[n]

    return result


def fitness(state: tuple[list, int]):
    """Number of games won"""

    result = 0
    formula = state[0]
    formula_result = state[1]

    for i in range(NUM_GAMES):
        nim = Nim(NUM_ROWS)
        player = 0
        while nim:
            if player == 0:
                ply = optimal(nim)
            elif player == 1:
                ok = False
                times = 0
                while not ok:
                    row = random.choice(range(NUM_ROWS))
                    if nim.rows[row] == 0:
                        continue

                    num = random.choice(range(1, nim.rows[row] + 1))
                    print (row, num, nim.rows[row])
                    c_result = check_result(formula, nim, row, num)

                    if c_result != None and c_result % 2 == formula_result:
                        ok = True
                    
                    times += 1
                    if times > NUM_ROWS**2:
                        break
                
                if times > NUM_ROWS**2:
                    break
                ply = Nimply(row, num)
                

            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            result += 1

    return result

In [89]:
@dataclass
class Individual:
    genotype: tuple[list, int]
    fitness: int

In [90]:
population = [
    Individual(
        genotype=(
            [
                random.choice(F) if i % 2 != 0 else random.randint(0, NUM_ROWS - 1)
                for i in range(random.randint(1, NUM_ROWS) * 2 - 1)
            ],
            random.choice([0, 1]),
        ),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)
    print(i)

population

Assert:  0 1 1
Assert:  1 1 3
Assert:  1 2 3
Assert:  1 3 3
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  3 1 7
Assert:  3 2 7
Assert:  3 3 7
Assert:  3 4 7
Assert:  3 5 7
Assert:  3 6 7
Assert:  3 7 7
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:  4 5 9
Assert:  4 6 9
Assert:  4 7 9
Assert:  4 8 9
Assert:  4 9 9
Assert:  3 2 7
4 9 9
Assert:  4 9 9
0 1 1
Assert:  0 1 1
Assert:  0 1 1
Assert:  1 1 3
Assert:  1 2 3
Assert:  1 3 3
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  3 1 5
Assert:  3 2 5
Assert:  3 3 5
Assert:  3 4 5
Assert:  3 5 5
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:  4 5 9
Assert:  4 6 9
Assert:  4 7 9
Assert:  4 8 9
Assert:  4 9 9
Assert:  1 2 3
3 4 5
Assert:  3 4 5
Assert:  3 4 5
Assert:  1 1 1
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  3 1 1
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:

[Individual(genotype=([1, 'div', 4], 0), fitness=10),
 Individual(genotype=([2, 'div', 1, '-', 3], 0), fitness=10),
 Individual(genotype=([3], 0), fitness=10),
 Individual(genotype=([2, '+', 0], 0), fitness=4),
 Individual(genotype=([1, 'and', 1, '-', 0, 'mod', 0, 'and', 3], 0), fitness=10)]

In [91]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    
    
    genome = {"love_small": 0.5}


### Player 0

### Player 1

## Oversimplified match

In [92]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(NUM_ROWS)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=6)
INFO:root:status: <1 3 5 1 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 3 5 0 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 5 0 9>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 0 5 0 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 0 5 0 0>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 5 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 4 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 0 2 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 1 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 0 won!


Assert:  0 1 1
Assert:  1 1 3
Assert:  1 2 3
Assert:  1 3 3
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  3 1 7
Assert:  3 2 7
Assert:  3 3 7
Assert:  3 4 7
Assert:  3 5 7
Assert:  3 6 7
Assert:  3 7 7
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:  4 5 9
Assert:  4 6 9
Assert:  4 7 9
Assert:  4 8 9
Assert:  4 9 9
Assert:  3 6 7
Assert:  3 1 1
Assert:  0 1 1
Assert:  1 1 3
Assert:  1 2 3
Assert:  1 3 3
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:  4 5 9
Assert:  4 6 9
Assert:  4 7 9
Assert:  4 8 9
Assert:  4 9 9
Assert:  1 2 3
Assert:  1 1 1
Assert:  0 1 1
Assert:  2 1 5
Assert:  2 2 5
Assert:  2 3 5
Assert:  2 4 5
Assert:  2 5 5
Assert:  4 1 9
Assert:  4 2 9
Assert:  4 3 9
Assert:  4 4 9
Assert:  4 5 9
Assert:  4 6 9
Assert:  4 7 9
Assert:  4 8 9
Assert:  4 9 9
Assert:  4 9 9
Assert:  0 1 1
Assert:  2 1 5
Assert:  2 2 5
Assert:  2